In [1]:
import numpy as np
import spatial
import importlib
importlib.reload(spatial)
from polynomial import Polymonial, multiXPoly, addXPoly

def calculate_Lambda(lambdas, alpha):
    # Begin algorithm 1
    R = len(lambdas)
    Ss = []
    for k in range(0, R):
        P = np.exp(-1j * alpha * lambdas[k])
        Vs = []
        for l in range(0, R):
            if l != k: 
                P = P / (lambdas[k] - lambdas[l])
                Vs.append(Polymonial([-lambdas[l], 1]))
        V = multiXPoly(Vs)
        Ss.append(V.multiX(P))
    S = addXPoly(Ss)
    return S
def calculate_M(lambdas, alpha):
    # This return Lambda_0, Lambda_1, ...
    Lambdas = calculate_Lambda(lambdas, alpha).coeff
    M = np.zeros([len(Lambdas), len(Lambdas)], dtype = np.complex128)
    for i in range(0, len(Lambdas)):
        for j in range(0, len(Lambdas)):
            M[i, j] = np.conjugate(Lambdas[i]) * Lambdas[j]
    return M

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, np.conjugate(a.T), rtol=rtol, atol=atol)
    
# alpha = -np.pi / 2
# lambdas = [-1, 0, 1]
# S = calculate_Lambda(lambdas, alpha)
# print(S.coeff)

In [48]:
def upper_matrix(M):
    upper_elements = []
    for i in range(0, M.shape[0]):
        for j in range(i + 1, M.shape[0], 2):           
            upper_elements.append(M[i, j])
    return np.expand_dims(np.asarray(upper_elements[1:]), 1)
alpha = np.pi / 4
beta = np.pi / 2
gamma = np.pi / 3
# lambdas = [-1, 0, 1]
# lambdas = [-3/2, -1/2, 1/2, 3/2]
# lambdas = [-2, -1, 0, 1, 2]
# lambdas = [-3, -2, -1, 1, 2, 3]
lambdas = [-3, -2, -1, 0, 1, 2, 5]
Lambdas = calculate_Lambda(lambdas, alpha).coeff
delta_Malpha = calculate_M(lambdas, alpha) - calculate_M(lambdas, -alpha)
delta_Mbeta = calculate_M(lambdas, beta) - calculate_M(lambdas, -beta)
delta_Mgamma = calculate_M(lambdas, gamma) - calculate_M(lambdas, -gamma)

d1 = 1/2
d2 = (-np.sqrt(2) + 1) / 4

# T_alpha = upper_matrix(delta_Malpha)
# T_beta = upper_matrix(delta_Mbeta)
# T_gamma = upper_matrix(delta_Mgamma)


In [49]:
print(np.round(delta_Malpha, 3))
print(upper_matrix(delta_Malpha))

[[0.+0.000e+00j 0.-1.567e+00j 0.+0.000e+00j 0.+1.570e-01j 0.-1.000e-03j
  0.-4.000e-03j 0.+0.000e+00j]
 [0.+1.567e+00j 0.+0.000e+00j 0.-4.830e-01j 0.-1.000e-03j 0.+2.400e-02j
  0.+0.000e+00j 0.-0.000e+00j]
 [0.-0.000e+00j 0.+4.830e-01j 0.+0.000e+00j 0.-4.800e-02j 0.+0.000e+00j
  0.+1.000e-03j 0.-0.000e+00j]
 [0.-1.570e-01j 0.+1.000e-03j 0.+4.800e-02j 0.+0.000e+00j 0.-2.000e-03j
  0.-0.000e+00j 0.+0.000e+00j]
 [0.+1.000e-03j 0.-2.400e-02j 0.-0.000e+00j 0.+2.000e-03j 0.+0.000e+00j
  0.-0.000e+00j 0.+0.000e+00j]
 [0.+4.000e-03j 0.-0.000e+00j 0.-1.000e-03j 0.+0.000e+00j 0.+0.000e+00j
  0.+0.000e+00j 0.-0.000e+00j]
 [0.-0.000e+00j 0.+0.000e+00j 0.+0.000e+00j 0.-0.000e+00j 0.-0.000e+00j
  0.+0.000e+00j 0.+0.000e+00j]]
[[0.+1.56723054e-01j]
 [0.-3.73037340e-03j]
 [0.-4.83009237e-01j]
 [0.+2.43795679e-02j]
 [0.-3.94412228e-04j]
 [0.-4.83015140e-02j]
 [0.+1.14965805e-03j]
 [0.-2.43779627e-03j]
 [0.+3.94021694e-05j]
 [0.-5.79872410e-05j]
 [0.-9.30266715e-07j]]


In [50]:
Ts = []
deltas = []
thetas = []
dim_d = int(len(lambdas)**2/4) - 1
for i in range(0, dim_d):
    theta = np.random.uniform(0, 2*np.pi)
    delta = (calculate_M(lambdas, theta) - calculate_M(lambdas, -theta))
    thetas.append(theta)
    deltas.append(delta)
    Ts.append(upper_matrix(delta))

In [51]:
T = Ts[0]
for i in range(1, len(Ts)):
    T = np.hstack((T, Ts[i]))

In [52]:
T.shape

(11, 11)

In [53]:
from scipy.linalg import null_space

init_rcond = 1
while True:
    
    d = (null_space(T, rcond = init_rcond))
    if d.shape[1] != 1:
        init_rcond /= 10
    else:
        break

In [54]:
d

array([[ 0.00851626-0.j        ],
       [ 0.13746947-0.39748559j],
       [-0.23470929+0.56110087j],
       [-0.12495755+0.26298878j],
       [-0.0056081 -0.00106667j],
       [-0.22814515+0.33151906j],
       [ 0.23132273+0.08574373j],
       [ 0.25880592-0.2050326j ],
       [ 0.01104206-0.00194323j],
       [ 0.00464935+0.18990442j],
       [-0.01557136-0.00353034j]])

In [55]:
T @ d

array([[-1.04083409e-16+1.73472348e-17j],
       [ 2.16840434e-18+1.73472348e-18j],
       [ 1.24900090e-16+4.16333634e-17j],
       [ 5.03069808e-17-1.47451495e-17j],
       [-2.22261445e-18+1.35525272e-18j],
       [-5.89805982e-17+3.12250226e-17j],
       [ 1.63714528e-17-6.50521303e-19j],
       [-3.05745013e-17+7.80625564e-18j],
       [ 8.13151629e-20-1.07064965e-18j],
       [-4.90601483e-18-3.52365706e-19j],
       [-6.51368336e-19+1.04185053e-19j]])

In [56]:
sumMatrix = d[0] * deltas[0]

for i in range(1, len(Ts)):
    sumMatrix += d[i] * deltas[i]
print(np.round(sumMatrix, 3))

[[ 0.   +0.j    -0.088-0.005j  0.   -0.j    -0.   +0.j     0.   +0.j
   0.   +0.j    -0.   -0.j   ]
 [ 0.088+0.005j  0.   +0.j     0.   +0.j    -0.   -0.j     0.   -0.j
   0.   +0.j    -0.   +0.j   ]
 [-0.   +0.j    -0.   -0.j     0.   +0.j    -0.   +0.j    -0.   -0.j
   0.   -0.j    -0.   -0.j   ]
 [ 0.   -0.j     0.   +0.j     0.   -0.j     0.   +0.j    -0.   +0.j
  -0.   -0.j     0.   -0.j   ]
 [-0.   -0.j    -0.   +0.j     0.   +0.j     0.   -0.j     0.   +0.j
  -0.   -0.j     0.   +0.j   ]
 [-0.   -0.j    -0.   -0.j    -0.   +0.j     0.   +0.j     0.   +0.j
   0.   +0.j    -0.   +0.j   ]
 [ 0.   +0.j     0.   -0.j     0.   +0.j    -0.   +0.j    -0.   -0.j
   0.   -0.j     0.   +0.j   ]]


array([[ 0.00000000e+00+0.00000000e+00j,  8.96429478e-03-1.50421630e-02j,
        -2.89909118e-17+4.94037830e-17j, -5.55111512e-17+8.32667268e-17j,
         1.15637639e-18-1.94406034e-18j,  8.67361738e-18-1.56125113e-17j],
       [-8.96429478e-03+1.50421630e-02j,  0.00000000e+00+0.00000000e+00j,
         2.77555756e-17+8.32667268e-17j,  5.02938534e-18-1.36410686e-17j,
        -1.73472348e-18+0.00000000e+00j, -2.64518292e-19+6.36386340e-19j],
       [ 2.89909118e-17-4.94037830e-17j, -2.77555756e-17-8.32667268e-17j,
         0.00000000e+00+0.00000000e+00j,  6.24500451e-17-9.71445147e-17j,
         9.30046641e-19-1.92239006e-18j, -1.21430643e-17+1.73472348e-17j],
       [ 5.55111512e-17-8.32667268e-17j, -5.02938534e-18+1.36410686e-17j,
        -6.24500451e-17+9.71445147e-17j,  0.00000000e+00+0.00000000e+00j,
         1.47451495e-17-1.73472348e-17j,  7.27349338e-20-1.33629377e-19j],
       [-1.15637639e-18+1.94406034e-18j,  1.73472348e-18+0.00000000e+00j,
        -9.30046641e-19+1.92239006

In [79]:
alpha = np.pi / 4
beta = 3*np.pi / 4
lambdas = [-1, 0, 1]

delta_Malpha = calculate_M(lambdas, alpha) - calculate_M(lambdas, -alpha)
delta_Mbeta = calculate_M(lambdas, beta) - calculate_M(lambdas, -beta)

d1 = (np.sqrt(2) + 1)/(4*np.sqrt(2))
d2 = (-np.sqrt(2) + 1)/(4*np.sqrt(2))

print(d1*delta_Malpha + d2*delta_Mbeta)

[[0.+0.00000000e+00j 0.-5.00000000e-01j 0.+0.00000000e+00j]
 [0.+5.00000000e-01j 0.+0.00000000e+00j 0.+5.55111512e-17j]
 [0.+0.00000000e+00j 0.-5.55111512e-17j 0.+0.00000000e+00j]]
